In [30]:
import numpy as np

from scipy.io import loadmat

import pandas as pd

from numpy.linalg import eig

from scipy.stats import mode

from sklearn.metrics import balanced_accuracy_score

from sklearn.model_selection import StratifiedKFold

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import warnings

warnings.filterwarnings("ignore")

from sklearn.utils import shuffle

import os

import scipy.stats as stats

import math

# PCA

In [31]:
class PCA :
    
    
    
    
    # ---------------function to calculate eigen values and eigen vector for any matrix
    
    
    def eig_vector( self, X ) :
        
        
        
        # centralize
    
        mean = np.mean( X, 0 )
        
        X_stand = X - mean
        
        
    
        # calculate correlation matrix
    
        X_cov = np.corrcoef( np.transpose( X_stand ) )
        
        
    
        # find the eigenvalues and eigenvectors
    
        e, V = eig( X_cov )
        
        
    
        # sort eigen vector according to eigen values 
        
        idx = np.argsort( -e )

        e = e[idx]

        V = V[:,idx]
        
        m, n = V.shape
        
        return e, V 



    # ----------------projection of X--------------------
    
    
    def transformation( self, X, no_of_components ) :
        
        
        
        e, V = self.eig_vector( X )
        
        p = V[:, : no_of_components ]
        
        
    
        # project the original dataset
    
        mean = np.mean( X, 0 )
        
        X_stand = X - mean
    
        X_transform = np.dot( X_stand, p )
        
        return X_transform

# Selection Methods

## Conditional Number    ---      ( max( lamda ) / 10 ) < lamda

In [32]:
# function return number of components 

def conditional_number( X ) :
    
    pca = PCA()
    
    e, V = pca.eig_vector( X )
    
    e_max = e[0]
    
    condition = e_max / 10
    
    no_of_components = np.argmax( e < condition )
    
    if( no_of_components == 0 ) :
        
        return 1
    
    else :
        
        return no_of_components

## Kaiser rule --- ( lamda > 1 )

In [33]:
# function return number of components

def kaiser_rule( X ) :
    
    pca = PCA()
    
    e, V = pca.eig_vector( X )
    
    return np.argmax( e < 1 )

## Broken Stick rule

In [34]:
# function return number of components

def broken_stick( X ) :
    
    
    
    pca = PCA()
    
    e, V = pca.eig_vector( X )
    
    
    
    # Calculate the proportional variance
    
    propvar = e / sum( e )
    
    
    
    # calculate the expected length of the k-th longest segment
    
    p = np.size( e )
    
    g = np.zeros( ( p ) )
    
    k = 0
    
    while( k < p ) :
        
        i = k
        
        while( i < p ) :
            
            g[k] = g[k] + ( 1 / ( i + 1 ) )
            
            i = i + 1
            
        k = k + 1

    g = g / p     
    
    
    
    
    # In the Broken-Stick model, the individual percentages of variance of the components are compared with the values expected from the “broken stick” distribution. 
    # The two distributions are compared element-by-element, and first value d + 1 where the expected valueis larger than the observed value determines the dimension.

    no_of_components = np.argmax( propvar < g )
    
    if( no_of_components == 0 ) :
        
        return 1
    
    else :
        
        return no_of_components



# Models

## KNN

In [35]:
# K Nearest Neighbors Classification

class K_Nearest_Neighbors_Classifier() : 
    
    
    def __init__( self, K ) :
        
        self.K = K
        
    
    
    # Function to store training set
        
    def fit( self, X_train, Y_train ) :
        
        self.X_train = X_train
        
        self.Y_train = Y_train
        
        # no_of_training_examples, no_of_features
        
        self.m, self.n = X_train.shape
    
    
    
    # Function for prediction
        
    def predict( self, X_test ) :
        
        self.X_test = X_test
        
        # no_of_test_examples, no_of_features
        
        self.m_test, self.n = X_test.shape
        
        # initialize Y_predict
        
        Y_predict = np.zeros( self.m_test )
        
        for i in range( self.m_test ) :
            
            x = self.X_test[i]
            
            # find the K nearest neighbors from current test example
            
            neighbors = np.zeros( self.K )
            
            neighbors = self.find_neighbors( x )
            
            # most frequent class in K neighbors
            
            Y_predict[i] = mode( neighbors )[0][0]    
            
        return Y_predict
    
    
    
    # Function to find the K nearest neighbors to current test example
          
    def find_neighbors( self, x ) :
        
        # calculate all the euclidean distances between current test example x and training set X_train
        
        euclidean_distances = np.zeros( self.m )
        
        for i in range( self.m ) :
            
            d = self.euclidean( x, self.X_train[i] )
            
            euclidean_distances[i] = d
        
        # sort Y_train according to euclidean_distance_array and store into Y_train_sorted
        
        inds = euclidean_distances.argsort()
        
        Y_train_sorted = self.Y_train[inds]
        
        return Y_train_sorted[:self.K]
    
    
    
    # Function to calculate euclidean distance
            
    def euclidean( self, x, x_train ) :
        
        return np.sqrt( np.sum( np.square( x - x_train ) ) )

## Logistic Regression

In [36]:
# # Logistic Regression

class LogitRegression() :
    
    
    
    def __init__( self, learning_rate, iterations, threshold ) :        
        
        self.learning_rate = learning_rate        
        
        self.iterations = iterations
        
        self.threshold = threshold
        
        
          
    # Function for model training   
    
    def fit( self, X, Y ) :        
        
        # no_of_training_examples, no_of_features        
        
        self.m, self.n = X.shape        
        
        # weight initialization        
        
        self.W = np.zeros( self.n )        
        
        self.b = 0        
        
        self.X = X        
        
        self.Y = Y
          
        # gradient descent learning
                  
        for i in range( self.iterations ) :            
            
            self.update_weights()            
        
        return self
      
    
    
    # Helper function to update weights in gradient descent
      
    def update_weights( self ) :           
        
        A = 1 / ( 1 + np.exp( - ( self.X.dot( self.W ) + self.b ) ) )
          
        # calculate gradients        
        
        tmp = ( A - self.Y.T )        
        
        tmp = np.reshape( tmp, self.m )        
        
        dW = np.dot( self.X.T, tmp ) / self.m         
        
        db = np.sum( tmp ) / self.m 
          
        # update weights    
        
        self.W = self.W - self.learning_rate * dW    
        
        self.b = self.b - self.learning_rate * db
          
        return self
      
    
    
    # Hypothetical function  h( x ) 
      
    def predict( self, X ) :    
        
        Z = 1 / ( 1 + np.exp( - ( X.dot( self.W ) + self.b ) ) )        
        
        Y = np.where( Z > self.threshold, 1, 0 )        
        
        return Y

# Selecting Components for Databases

In [37]:
files = os.listdir('./Databases/' )


# Initialize lists to store each database name, no of attributes, cases and their respective dimensions  

Databases = []

Attributes = []

Cases = []

PCA_K = []

PCA_BS = []

PCA_CN = []


for file in files :
    
    
    # read one database at a time
    
    data = loadmat( './Databases/' + file )
    
    X = data['X']
    
    
    # standarise data
    
    X = stats.zscore( X )
    
    y = data['Y']
    
    
    try :
        
        
        # no. of components selected by kaiser, broken stick and condtional number
    
        
        # kaiser rule

        no_of_components_kaiser_rule = kaiser_rule( X )
        
        PCA_K.append( no_of_components_kaiser_rule )
    
        
        # broken stick
    
        no_of_components_broken_stick_rule = broken_stick( X )
    
        PCA_BS.append( no_of_components_broken_stick_rule )
    
    
        
        # conditional number

        no_of_components_conditonal_number = conditional_number( X )
    
        PCA_CN.append( no_of_components_conditonal_number )
        
        
        # get shapes and append them in their lists
    
        ( m, n ) = X.shape
    
        Databases.append( file )
    
        Attributes.append( n )
    
        Cases.append( m )


    
    except :
        
         continue

In [38]:
Component_Selection = { 'Databases' : Databases, 'Attributes' : Attributes, 'Cases' : Cases, 'PCA-K' : PCA_K, 'PCA-BS' : PCA_BS, 
    
         'PCA-CN' : PCA_CN
     
    }

Component_Selection = pd.DataFrame( Component_Selection )

Component_Selection.to_csv( 'Component_Selection.csv' )

Component_Selection

,Databases,Attributes,Cases,PCA-K,PCA-BS,PCA-CN
0,minboone.mat,50,130064,4,1,1
1,spect.mat,22,267,7,3,12
2,Banknote.mat,4,1372,2,2,3
3,liver.mat,10,579,4,1,7
4,vertebral.mat,6,310,2,1,5
5,skin.mat,3,245057,1,1,2
6,diabetic.mat,19,1151,5,3,8
7,sonar.mat,60,208,13,6,11
8,spectf.mat,44,267,10,3,6
9,Musk2.mat,166,6598,25,13,6


# Modelling and Balanced Accuracy calculation 10 times

In [39]:
# return the average of balanced accuracy after running 10 times with 10 fold stratified cross-validation

def magic( X, y, model ) :
    
    
    
    # outer loop to calculate the balanced accuracy 10 times
    
    balanced_accuracies = []
    
    
    
    for i in range( 0, 10 ) :
        
        
        # shuffle X, y before Splitting
        
        shuffle( X, y )
        
        skfold = StratifiedKFold( n_splits = 10, shuffle = True )

        balanced_accuracy_K_folds = []
        
        
        
        # inner loop for 10 fold stratified cross validation

        for train_index, test_index in skfold.split( X, y ) :
            
            X_train, X_test = X[train_index], X[test_index]
    
            y_train, y_test = y[train_index], y[test_index]
         
            model.fit( X_train, y_train )
    
            balanced_accuracy_K_folds.append( balanced_accuracy_score( y_test, model.predict( X_test ) ) )
             
        
             
        balanced_accuracies.append( np.mean( balanced_accuracy_K_folds ) )

        
    return np.mean( balanced_accuracies )

# KNN Results

In [40]:
Thres = pd.read_csv( 'Thres.csv' )

Component_Selection = pd.read_csv( 'Component_Selection.csv' )

Databases = Thres["Databases"]

In [41]:
Databases = Databases[:10]

In [42]:
BA_K = []

BA_BS = []

BA_CN = []


for file in Databases :

    
    
    # Get X and Y 
    
    data = loadmat( 'Databases/' + file )

    X = data['X']
    
    
    # standarise data
    
    X = stats.zscore( X )
    
    

    y = data['Y']
    
    
    
    
    # select row from Component_Selection dataframe
    
    x = Component_Selection.loc[ Component_Selection[ 'Databases' ] == file ]
    
    
    
    
    
    # For Kaiser-rule
    
    no_of_components_kaiser_rule = x["PCA-K"].values[0]
    
    pca = PCA()

    X_kaiser_rule = pca.transformation( X, no_of_components_kaiser_rule )
    
    knn = K_Nearest_Neighbors_Classifier( K = 3 )

    BA_K.append( magic( X_kaiser_rule, y, knn ) )
    
    
    
    
    # For Broken Stick
    
    no_of_components_broken_stick = x["PCA-BS"].values[0]
    
    pca = PCA()

    X_broken_stick = pca.transformation( X, no_of_components_broken_stick )
    
    knn = K_Nearest_Neighbors_Classifier( K = 3 )

    BA_BS.append( magic( X_broken_stick, y, knn ) )
    
    
    
    
    # For conditional number
    
    no_of_components_conditonal_number = x["PCA-CN"].values[0]
    
    pca = PCA()

    X_conditional_number = pca.transformation( X, no_of_components_conditonal_number )
    
    knn = K_Nearest_Neighbors_Classifier( K = 3 )

    BA_CN.append( magic( X_conditional_number, y, knn ) )    

In [43]:
Databases = Databases.to_list()

In [44]:
KNN_Results =  { 'Databases' : Databases, 'BA_K' : BA_K, 'BA_BS' : BA_BS, 'BA_CN' : BA_CN }

KNN_Results = pd.DataFrame( KNN_Results )

KNN_Results.to_csv( 'KNN_Results.csv' )

In [45]:
KNN_Results

,Databases,BA_K,BA_BS,BA_CN
0,spect.mat,0.638615,0.646448,0.646487
1,Banknote.mat,0.862075,0.860333,0.978551
2,liver.mat,0.618446,0.546126,0.560894
3,vertebral.mat,0.635024,0.617762,0.784571
4,diabetic.mat,0.574084,0.600578,0.608671
5,sonar.mat,0.876967,0.850737,0.863980
6,spectf.mat,0.678409,0.584017,0.663784
7,Musk.mat,0.885202,0.858719,0.829500
8,Immunotherapy.mat,0.521786,0.542768,0.552232
9,Cryotherapy.mat,0.793250,0.800500,0.791750


# Logistic Regression Results

In [46]:
BA_K = []

BA_BS = []

BA_CN = []


for file in Databases :

    
    
    # Get X and Y 
    
    data = loadmat( 'Databases/' + file )

    X = data['X']
    
    
    # standarise data
    
    X = stats.zscore( X )
    

    y = data['Y']
    
    
    
    
    # select row from Component_Selection dataframe
    
    x = Component_Selection.loc[ Component_Selection[ 'Databases' ] == file ]
    
    
    
    
    # select row from Component_Selection dataframe
    
    z = Thres.loc[ Thres["Databases"] == file ]
    
    
    
    best_thres = z["Thresholds"].values[0]
    
    if( math.isnan( best_thres ) ) :
        
        best_thres = 0.5
        

    
    
    
    # For Kaiser-rule
    
    no_of_components_kaiser_rule = x["PCA-K"].values[0]
    
    pca = PCA()

    X_kaiser_rule = pca.transformation( X, no_of_components_kaiser_rule )
    
    model = LogitRegression( 0.01, 500, best_thres )

    BA_K.append( magic( X_kaiser_rule, y, model ) )
    
    
    
    
    # For Broken Stick
    
    no_of_components_broken_stick = x["PCA-BS"].values[0]
    
    pca = PCA()

    X_broken_stick = pca.transformation( X, no_of_components_broken_stick )
    
    model = LogitRegression( 0.01, 500, best_thres )

    BA_BS.append( magic( X_broken_stick, y, model ) )
    
    
    
    
    # For conditional number
    
    no_of_components_conditonal_number = x["PCA-CN"].values[0]
    
    pca = PCA()

    X_conditional_number = pca.transformation( X, no_of_components_conditonal_number )
    
    model = LogitRegression( 0.01, 500, best_thres )
    
    BA_CN.append( magic( X_conditional_number, y, model ) )    

In [47]:
Logistic_Results =  { 'Databases' : Databases, 'BA_K' : BA_K, 'BA_BS' : BA_BS, 'BA_CN' : BA_CN }

Logistic_Results = pd.DataFrame( Logistic_Results )

Logistic_Results.to_csv( 'Logistic_Results.csv' )

In [48]:
Logistic_Results

,Databases,BA_K,BA_BS,BA_CN
0,spect.mat,0.733528,0.745840,0.743978
1,Banknote.mat,0.751120,0.751828,0.847754
2,liver.mat,0.503383,0.516451,0.509786
3,vertebral.mat,0.654357,0.656881,0.745762
4,diabetic.mat,0.598709,0.594862,0.610058
5,sonar.mat,0.776462,0.747717,0.776455
6,spectf.mat,0.644734,0.553660,0.632989
7,Musk.mat,0.785459,0.710235,0.673094
8,Immunotherapy.mat,0.562321,0.500000,0.500000
9,Cryotherapy.mat,0.817250,0.805000,0.806750


In [50]:
# while( True ) :
    
#     code()
    
#     eat()
    
#     sleep()

# Fisher LDA Results